### Imports

In [1]:
# Scinet modules
from scinet import *

# My custom modules
import scinet.ed_stokes as sto

#Other modules
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np

%load_ext tensorboard
import tensorflow as tf
import datetime, os


### Helper Functions (based on oscillator example)

In [2]:
def gen_input_stokes(Fr, St, t_observ, tt_predicted):
    in1 = np.array([sto.stokes_eqn(Fr, St, t_observ) for _ in tt_predicted])
    in2 = np.reshape(tt_predicted, (-1, 1))
    out = in2 #dummy filler
    return [in1, in2, out]

def stokes_representation_plot_flat(net, Fr_range, St_range, t_observ, step_num=100, eval_time=1.5):
    Fr_vec = np.linspace(*Fr_range, num=step_num)
    St_vec = np.logspace(*St_range, num=step_num)
    
    FR, ST = np.meshgrid(Fr_vec, St_vec)
    
    St2_Fr_vec = (St_vec**2) * Fr_vec
    inv_St_vec = np.flip(1.0/St_vec)
    
    ST2_FR, INV_ST = np.meshgrid(St2_Fr_vec, inv_St_vec)
    
    eval_time = np.array([eval_time])

    maxColor = 0
    minColor = 0
    
    #St - Fr axes
    out = np.array([net.run(gen_input_stokes(Fr, St, t_observ, eval_time), net.mu)[0] 
                    for Fr, St in zip(np.ravel(FR), np.ravel(ST))])
    
    for i in range(net.latent_size):
        tempz = out[:, i]
        maxColor = max(np.max(tempz), maxColor)
        minColor = min(np.min(tempz), minColor)

    for i in range(net.latent_size):
        zs = out[:, i]
        Z = np.reshape(zs, FR.shape)
    
        fig = plt.figure(i+100*net.latent_size, figsize=(5, 4))
        plt.contourf(FR, ST, Z, 20, cmap=cm.inferno)
        plt.clim([minColor,maxColor])
        plt.colorbar()        
        
        plt.xlabel(r'$Fr$',fontsize='x-large')
        plt.ylabel(r'$St$',fontsize='x-large')
        plt.xscale('log')
        plt.yscale('log')
        plt.title('Latent activation {}/{}'.format(i + 1,net.latent_size),fontsize='x-large')
        
        plt.tight_layout()
        
        #plt.figure()
        #plt.contourf(ST**2 * FR, 1/ST, Z, 20, cmap=cm.inferno)
        #plt.clim([minColor,maxColor])
        #plt.colorbar()
        
        #plt.xlabel(r'$St^2 Fr$')
        #plt.ylabel(r'$1/St$')
        #plt.xscale('log')
        #plt.yscale('log')
        #plt.xlim([np.min(ST2_FR),np.max(ST2_FR)])
        #plt.ylim([np.min(INV_ST),np.max(INV_ST)])        
        #plt.title('Latent activation {}/{}'.format(i + 1,net.latent_size))
        
        plt.tight_layout()
    

def plot_stokes_prediction(net_, Fr_, St_, t_observ, t_predict):    
    x_correct = sto.stokes_eqn(Fr, St, t_predict)
    x_predict = net_.run(gen_input_stokes(Fr_, St_, t_observ, t_predict), net_.output).ravel()
    fig = plt.figure(figsize=(7, 4))
    ax = fig.add_subplot(111)
    ax.plot(t_predict, x_correct, color=orange_color, label='True time evolution')
    ax.plot(t_predict, x_predict, '--', color=blue_color, label='Predicted time evolution')
    ax.set_xlabel(r'$t$ [$s$]')
    ax.set_ylabel(r'$x$ [$m$]')
    handles, labels = ax.get_legend_handles_labels()
    lgd=ax.legend(handles, labels,loc='upper left', bbox_to_anchor=(0.6, 1.3), shadow=True, ncol=1)
    fig.tight_layout()
    return fig

def plot_triplet(net_, Fr_, St_, t_o, t_full, t_q):    
    x_full = sto.stokes_eqn(Fr_, St_, t_full)
    x_o    = sto.stokes_eqn(Fr_, St_, t_o)
    x_a    = sto.stokes_eqn(Fr_, St_, t_q)
    
    fig = plt.figure(figsize=(5, 2.5))
    ax = fig.add_subplot(111)
    
    ax.plot(t_full, x_full, '-', color=orange_color, label='True time evolution')
    ax.plot(t_o, x_o, '.', color=blue_color, label='Observations')
    ax.plot(t_q, x_a, '*', color=green_color, label='Question-Answer pair', markersize=15)
    
    #ax.plot(t_predict, x_predict, '--', color=blue_color, label='Predicted time evolution')
    ax.set_xlabel(r'$t$')
    ax.set_ylabel(r'$x$')
    ax.set_title('$Fr=${}, $St=${}'.format(Fr_,St_))
    handles, labels = ax.get_legend_handles_labels()
    lgd=ax.legend(handles, labels,loc='upper left', bbox_to_anchor=(0.02, 0.55), shadow=True, ncol=1)
    fig.tight_layout()
    return fig

def plot_errors(nets, Fr_range, St_range, t_observ, step_num=100, eval_time=1.5):
    
    maxColor = 0
    minColor = 0
    
    for ind, net in enumerate(nets):
        Fr_vec = np.linspace(*Fr_range, num=step_num)
        St_vec = np.logspace(*St_range, num=step_num)

        FR, ST = np.meshgrid(Fr_vec, St_vec)
        eval_time = np.array([eval_time])

        error = np.array([
                    np.abs(
                    net.run(gen_input_stokes(Fr, St, t_observ, eval_time), net.output) 
                   -sto.stokes_eqn(Fr, St, t_q)
                    )
                    for Fr, St in zip(np.ravel(FR), np.ravel(ST))
                    ])

        Z = np.reshape(error, FR.shape)
        
        maxColor = max(np.max(error), maxColor)
        minColor = min(np.min(error), minColor)

        fig = plt.figure(ind,figsize=(5, 4))
        plt.pcolor(FR, ST, Z, cmap=cm.inferno)

    for ind, net in enumerate(nets):
        plt.figure(ind)
        plt.colorbar()        
        plt.clim([minColor,maxColor])
        plt.xlabel(r'$Fr$',fontsize='x-large')
        plt.ylabel(r'$St$',fontsize='x-large')
        plt.yscale('log')
        if net.latent_size == 1:
            plt.title('Error, {} latent neuron'.format(net.latent_size),fontsize='x-large')        
        else:
            plt.title('Error, {} latent neurons'.format(net.latent_size),fontsize='x-large')
        
        plt.tight_layout()

In [3]:
blue_color='#000cff'
orange_color='#ff7700'
green_color='#009c29'

### Create Test Data

In [4]:
num_examples = 200000
dev_percent = 5
t_sim = np.linspace(0,1,50)

sto.stokes_data(num_examples, t_sample=t_sim, fileName='stokes_example');
td, vd, ts, vs, proj = dl.load(dev_percent, 'stokes_example')

### Load Networks

In [10]:
filename2Load_1latent = 'stokesNet_1latent_2021-03-14'
filename2Load_2latent = 'stokesNet_2latent_2021-03-14'
filename2Load_3latent = 'stokesNet_3latent_2021-03-13'
filename2Load_4latent = 'stokesNet_4latent_2021-03-14'
net1 = nn.Network.from_saved(filename2Load_1latent);
net2 = nn.Network.from_saved(filename2Load_2latent);
net3 = nn.Network.from_saved(filename2Load_3latent);
net4 = nn.Network.from_saved(filename2Load_4latent);

train_losses1 = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')
dev_losses1   = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')
train_losses2 = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')
dev_losses2   = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')
train_losses3 = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')
dev_losses3   = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')
train_losses4 = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')
dev_losses4   = np.load(io.tf_save_path + 'stokesNet_1latent_2021-03-14.npy')

{'decoder_num_units': [100, 100], 'input2_size': 1, 'tot_epochs': 2000, 'latent_size': 1, 'output_size': 1, 'encoder_num_units': [500, 100], 'input_size': 50, 'load_file': 'stokesNet_1latent_2021-03-14', 'name': 'StokesNet'}
INFO:tensorflow:Restoring parameters from /home/jrwest/Courses/Winter_2021/CS230/project/nn_physical_concepts_cs230/tf_save/stokesNet_1latent_2021-03-14.ckpt
Loaded network from file stokesNet_1latent_2021-03-14
{'decoder_num_units': [100, 100], 'input2_size': 1, 'tot_epochs': 2000, 'latent_size': 2, 'output_size': 1, 'encoder_num_units': [500, 100], 'input_size': 50, 'load_file': 'stokesNet_2latent_2021-03-14', 'name': 'StokesNet'}
INFO:tensorflow:Restoring parameters from /home/jrwest/Courses/Winter_2021/CS230/project/nn_physical_concepts_cs230/tf_save/stokesNet_2latent_2021-03-14.ckpt
Loaded network from file stokesNet_2latent_2021-03-14
{'decoder_num_units': [100, 100], 'input2_size': 1, 'tot_epochs': 2000, 'latent_size': 3, 'output_size': 1, 'encoder_num_units

### Plot Stuff

In [22]:
# Plot losses
%matplotlib tk
fig = plt.figure(1,figsize=(6, 3))

plt.plot(np.array(train_losses2), 'b-')
plt.plot(np.array(dev_losses2), 'r-')
plt.xlabel('# Epochs',fontsize='large')
plt.legend(['training loss','dev loss'],fontsize='large')
plt.yscale('log')

fig.tight_layout()

In [80]:
# Plot example trajectories
# show observations, question, and answer
%matplotlib tk
Fr = 1.0
St = 100
t_predict = np.linspace(0, 2.0, 250)
t_plot = np.linspace(0,1,20)

plot_triplet(net1, 1, 90, t_plot, t_predict, 1.8);
plot_triplet(net1, 5, 0.3,     t_plot, t_predict, 0.3);

In [90]:
# Plot loss as function of latent size
%matplotlib tk
num_latent = np.array([1,2,3,4])
recon_losses = np.array([net1.run(vd, net1.recon_loss), 
                         net2.run(vd, net2.recon_loss), 
                         net3.run(vd, net3.recon_loss), 
                         net4.run(vd, net4.recon_loss)])
kl_losses      = np.array([net1.run(vd, net1.kl_loss), 
                         net2.run(vd, net2.kl_loss),
                         net3.run(vd, net3.kl_loss), 
                         net4.run(vd, net4.kl_loss)])

fig = plt.figure(1,figsize=(5, 3))
plt.bar(num_latent, recon_losses)
plt.xticks(num_latent, num_latent)
plt.xlabel('# Latent Neurons',fontsize='x-large')
plt.ylabel('Reconstruction Loss',fontsize='x-large')
fig.tight_layout()

fig = plt.figure(2,figsize=(5, 3))
plt.bar(num_latent, kl_losses)
plt.xticks(num_latent, num_latent)
plt.xlabel('# Latent Neurons',fontsize='x-large')
plt.ylabel('KL Loss',fontsize='x-large')
fig.tight_layout()

In [132]:
# Plot error as fcn of St and Fr for 1 and 2 neuron nets
%matplotlib tk
step_num = 100    
plot_errors([net1, net2], [0.01,5.0], [-2,2], t_sim, eval_time=1.5, step_num=step_num)    

In [147]:
# Plot activations for 1, 2, and 3 nets (pcolor)
%matplotlib tk
stokes_representation_plot_flat(net1, [0.01,5.0], [-2,2], t_sim, eval_time=1.5, step_num=100);
stokes_representation_plot_flat(net2, [0.01,5.0], [-2,2], t_sim, eval_time=1.5, step_num=100);
stokes_representation_plot_flat(net3, [0.01,5.0], [-2,2], t_sim, eval_time=1.5, step_num=100);